In [127]:
import os
import cv2
from shutil import copyfile
import matplotlib.pyplot as plt
import csv
import random
import cv2
import numpy as np
from skimage import data
from skimage import exposure
from skimage.transform import match_histograms

In [128]:
root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/histogram/image/"
image_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/image/"
mask_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/mask/"
box_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/box/"
realbox_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_unity/real_boxes/"
realbox_list = os.listdir(realbox_root)
box_list = os.listdir(box_root)

In [129]:
for box in box_list:
    mask_name = box.split('_original.png')[0] + '_seg.png'
    fake_img = cv2.imread(os.path.join(box_root, box))
    real_img = cv2.imread(os.path.join(realbox_root, realbox_list[random.randint(0,len(realbox_list)-1)]))
    matched = match_histograms(fake_img, real_img, multichannel=True)
    image = cv2.imread(os.path.join(image_root, box))
    mask_img = cv2.imread(os.path.join(mask_root, mask_name), 0)
    mask_img[mask_img > 0] = 255
    mask = cv2.cvtColor(mask_img, cv2.COLOR_GRAY2RGB)
    matched = cv2.bitwise_and(matched, mask, mask=mask_img)
    mask_img[mask_img < 255] = 1
    mask_img[mask_img == 255] = 0
    mask_img[mask_img == 1] = 255
    mask = cv2.cvtColor(mask_img, cv2.COLOR_GRAY2RGB)
    image = cv2.bitwise_and(image, mask, mask=mask_img)
    result = cv2.add(matched, image)
    cv2.imwrite(os.path.join(root, box), result)